# New Training

In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Paths
LABELS_PATH = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/label_to_index.json"
DATA_DIR = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints"
MODEL_PATH = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/best_lstm_model.pth"

# Parameters
SEQUENCE_LENGTH = 102  # Number of frames per sequence
INPUT_SIZE = 225  # Adjusted to include face keypoints
BATCH_SIZE = 32
EPOCHS = 500
LR = 0.0001
PATIENCE = 5  # Early stopping patience

# Load labels
with open(LABELS_PATH, "r") as f:
    label_to_index = json.load(f)
    index_to_label = {v: k for k, v in label_to_index.items()}
    NUM_CLASSES = len(label_to_index)

# Dataset class
class ASLDataset(Dataset):
    def __init__(self, data_dir, labels_map, sequence_length):
        self.data_dir = data_dir
        self.sequence_length = sequence_length
        self.samples = []
        
        for file in os.listdir(data_dir):
            if file.endswith(".npy"):
                filename_parts = file.split("_")
                label_name = filename_parts[1]  # Extract label from filename structure
                if label_name in labels_map:
                    self.samples.append((os.path.join(data_dir, file), labels_map[label_name]))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        keypoints = np.load(file_path)
        
        # Ensure all sequences are of the same length
        if keypoints.shape[0] < self.sequence_length:
            pad = np.zeros((self.sequence_length - keypoints.shape[0], keypoints.shape[1]))
            keypoints = np.vstack((keypoints, pad))
        else:
            keypoints = keypoints[:self.sequence_length]
        
        return torch.tensor(keypoints, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
    
# LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])  # Apply Dropout
        out = self.fc(out)
        return out

# Load data
dataset = ASLDataset(DATA_DIR, label_to_index, SEQUENCE_LENGTH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize model
model = LSTMModel(input_size=INPUT_SIZE, hidden_size=512, num_layers=3, num_classes=NUM_CLASSES)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Apply Xavier Initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param.data)

model.apply(init_weights)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# Early stopping
best_loss = float("inf")
stopping_counter = 0

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for keypoints, labels in dataloader:
        keypoints, labels = keypoints.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(keypoints)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / float(total)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Adjust learning rate
    scheduler.step(avg_loss)
    
    # Save best model if it improves
    if avg_loss < best_loss:
        best_loss = avg_loss
        stopping_counter = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"✅ Best model saved at epoch {epoch+1} with loss {avg_loss:.4f}")
    else:
        stopping_counter += 1
        if stopping_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

print("Training completed! Best model saved at:", MODEL_PATH)

Epoch [1/500], Loss: 5.2043, Accuracy: 0.93%
✅ Best model saved at epoch 1 with loss 5.2043
Epoch [2/500], Loss: 5.0715, Accuracy: 1.15%
✅ Best model saved at epoch 2 with loss 5.0715
Epoch [3/500], Loss: 4.9832, Accuracy: 1.21%
✅ Best model saved at epoch 3 with loss 4.9832
Epoch [4/500], Loss: 4.9157, Accuracy: 1.78%
✅ Best model saved at epoch 4 with loss 4.9157
Epoch [5/500], Loss: 4.9164, Accuracy: 1.71%
Epoch [6/500], Loss: 4.8909, Accuracy: 1.65%
✅ Best model saved at epoch 6 with loss 4.8909
Epoch [7/500], Loss: 4.7905, Accuracy: 1.87%
✅ Best model saved at epoch 7 with loss 4.7905
Epoch [8/500], Loss: 4.7034, Accuracy: 2.69%
✅ Best model saved at epoch 8 with loss 4.7034
Epoch [9/500], Loss: 4.6477, Accuracy: 2.98%
✅ Best model saved at epoch 9 with loss 4.6477
Epoch [10/500], Loss: 4.9025, Accuracy: 1.95%
Epoch [11/500], Loss: 4.6362, Accuracy: 2.94%
✅ Best model saved at epoch 11 with loss 4.6362
Epoch [12/500], Loss: 4.5678, Accuracy: 3.44%
✅ Best model saved at epoch 12 wi

In [ ]:
# Testing cell
def test_model():
    """Load the trained model and test on a random sample from the dataset."""
    model.load_state_dict(torch.load("/home/haggenmueller/asl_detection/machine_learning/models/lstm/no_batch_norm_lstm_model.pth"))
    model.eval()
    
    sample, label = dataset[0]
    sample = sample.unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(sample)
        predicted_class = torch.argmax(output, dim=1).item()
    
    print(f"Actual Label: {index_to_label[label.item()]}")
    print(f"Predicted Label: {index_to_label[predicted_class]}")

test_model()

/tmp/ipykernel_2930592/3936693878.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


FileNotFoundError: [Errno 2] No such file or directory: '/home/haggenmueller/asl_detection/machine_learning/models/lstm/best_lstm_model.pth'